# Load data from MOST .nc file

Sample of how to load data from a MOST simulation of Seaside, OR.

Then extract and plot:
 - time series at specified gauge locations, 
 - inundation maps at specified times

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import netCDF4, xarray
from clawpack.visclaw import geoplot, colormaps
from scipy.interpolate import RegularGridInterpolator

In [ ]:
ncfile = '/Users/rjl/D/seaside_tsunami_data.nc'

In [ ]:
ncdata = xarray.open_dataset(ncfile, decode_timedelta=False)
#ncdata = netCDF4.Dataset(ncfile, 'r')  # alternative way, but xarray gives nicer wrapper

In [ ]:
ncdata

## extract as numpy arrays and transpose so [i,j] index corresponds to [x,y]:

In [ ]:
t = asarray(ncdata.variables['time'])
x = asarray(ncdata.variables['lon'])
y = asarray(ncdata.variables['lat'])

# transpose multidimensional arrays [i,j,k] index corresponds to [x,y,t]:
B = asarray(ncdata.variables['bathy']).T
h = asarray(ncdata.variables['h']).T
depth = asarray(ncdata.variables['depth']).T
speed = asarray(ncdata.variables['speed']).T

X,Y = meshgrid(x,y,indexing='ij')
tmin = t / 60. # convert to minutes

In [ ]:
print('t has %i times with increment %.4f seconds' % (len(t),(t[1]-t[0])))
print('Spatial arrays on grid with %i longitudes, %i latitudes' % (len(x),len(y)))
print('Shapes: ', t.shape, x.shape, y.shape, B.shape, h.shape)

In [ ]:
ncdata.close()

## make interpolators for evaluating at arbitray x,y,t:

In [ ]:
h_fcn = RegularGridInterpolator((x,y,t), h)
depth_fcn = RegularGridInterpolator((x,y,t), depth)
speed_fcn = RegularGridInterpolator((x,y,t), speed)
B_fcn = RegularGridInterpolator((x,y), B)

## Plotting stuff:

In [ ]:
land_cmap = colormaps.make_colormap({ 0.0:[0.1,0.4,0.0],
                                     0.25:[0.0,1.0,0.0],
                                      0.5:[0.8,1.0,0.5],
                                      1.0:[0.8,0.5,0.2]})
sea_cmap = plt.get_cmap('Blues_r')

cmap, norm = colormaps.add_colormaps((land_cmap, sea_cmap),
                                     data_limits=[-1,1],
                                     data_break=0)

In [ ]:
def plot_topo():
    fig,ax = subplots(figsize=(6,8))
    pcolormesh(X,Y,B,cmap=cmap)
    clim(-30,30)
    colorbar(extend='both',shrink=0.8,label='meters')
    ax.set_aspect(1/cos(pi*y.mean()/180))
    ticklabel_format(useOffset=False)
    xticks(rotation=20)
    title('Bathymetry / topography relative to MHW')

def plot_topo_contours(clines=[0.]):
    fig,ax = subplots(figsize=(6,8))
    contour(X,Y,B,clines,colors='g',linewidths=0.8)
    ax.set_aspect(1/cos(pi*y.mean()/180))
    ticklabel_format(useOffset=False)
    xticks(rotation=20)
    #title('Bathymetry / topography relative to MHW')

## Code to extract a time series at a point:

In [ ]:
def extract_gauge(xg, yg, make_plots=True):
    """
    Extract times series at a gauge location xg,yg
    """

    h_series = h_fcn((xg,yg,t))
    depth_series = depth_fcn((xg,yg,t))
    speed_series = speed_fcn((xg,yg,t))
    Bg = B_fcn((xg,yg))

    if make_plots:
    
        plot_topo()
        plot([xg], [yg], 'rx')
        text(xg+0.001, yg, 'Gauge', ha='left', color='r')
        
        figure(figsize=(8,6))
        subplot(211)
        plot(tmin, h_series, 'b', label='surface elevation')
        plot(tmin, depth_series, 'r', label='depth')
        legend(loc='upper left', framealpha=1)
        grid(True)
        xlabel('minutes')
        ylabel('meters')
        title('B = %.2fm at x = %.4f, y = %.4f' % (Bg,xg,yg));
    
        subplot(212)
        plot(tmin, speed_series, 'm', label='speed')
        grid(True)
        legend(loc='upper left', framealpha=1)
        xlabel('minutes')
        ylabel('meters / second')
        tight_layout()

    return t, h_series, depth_series, speed_series

In [ ]:
t, h_series, depth_series, speed_series = extract_gauge(-123.94, 46.005)

In [ ]:
t, h_series, depth_series, speed_series = extract_gauge(-123.93, 45.995)

## Code to extract a snapshot of depth at a give time:

In [ ]:
def extract_grid_onshore(tt, make_plots=True):
    """
    extract depth and speed at time tt (minutes) at onshore points (nan offshore)
    if make_plots:  also plot depth on contour plot of topo
    """
    depth_t = depth_fcn((X,Y,tt*60))
    depth_onshore = where(B>0, depth_t, nan)
    speed_t = speed_fcn((X,Y,tt*60))
    speed_onshore = where(B>0, speed_t, nan)
    if make_plots:
        
        cmap_reds = plt.get_cmap('Reds')
        plot_topo_contours()
        pcolormesh(X,Y,depth_onshore,cmap=cmap_reds)
        colorbar(extend='max',label='meters',shrink=0.8)
        title('Onshore depth at t = %.2f minutes' % tt)

    return depth_onshore, speed_onshore

In [ ]:
depth_onshore, speed_onshore = extract_grid_onshore(35)

In [ ]:
depth_onshore, speed_onshore = extract_grid_onshore(45)

## Create a smaller .nc file with only the surface elevation:

Illustrates how to write netcdf files using xarray.

In [ ]:
data_dict = {'eta': (('x','y','t'), h)}
coords = {'x':x, 'y':y, 't':t}
ds = xarray.Dataset(data_dict, coords)

In [ ]:
ds.to_netcdf('test_eta.nc')

In [ ]:
!du -hs test_eta.nc

### reload to test:

In [ ]:
ff = xarray.open_dataset('test_eta.nc')

In [ ]:
ff

In [ ]:
asarray(ff.variables['eta']).shape

In [ ]:
ff.close()